# Risk Measures

This file contains the examples in the docstrings as runnable code.

In [1]:
using PortfolioOptimiser, Clarabel, JuMP, StatsBase

# Randomly generated normally distributed returns.
ret = [0.670643    1.94045   -0.0896267   0.851535    -0.268234
       1.33575    -0.541003   2.28744    -0.157588    -1.45177
       -1.91694    -0.167745   0.920495    0.00677243  -1.29112
       0.123141    1.59841   -0.185076    2.58911     -0.250747
       1.92782     1.01679    1.12107     1.09731     -0.99954
       2.07114    -0.513216  -0.532891    0.917748    -0.0346682
       -1.37424    -1.35272   -0.628216   -2.76234     -0.112378
       1.3831      1.14021   -0.577472    0.224504     1.28137
       -0.0577619  -0.10658   -0.637011    1.70933      1.84176
       1.6319      2.05059   -0.21469    -0.640715     1.39879];

# Instantiate portfolio instance.
port = Portfolio(; ret = ret, assets = 1:size(ret, 2),
                 solvers = Dict(:Clarabel => Dict(:solver => Clarabel.Optimizer,
                                                  :check_sol => (allow_local = true,
                                                                 allow_almost = true),
                                                  :params => Dict("verbose" => false))));
# Compute asset statistics.
asset_statistics!(port)
# Clusterise assets (for hierarchical optimisations).
cluster_assets!(port)

## Standard Deviation, `SD`

Standard deviation.

In [2]:
rm = SD()

SD(RMSettings{Float64, Float64}(true, 1.0, Inf), nothing)

Optimise portfolio.

In [3]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.42346e-10
2,2,0.018383
3,3,0.520498
4,4,0.0340474
5,5,0.427072


Compute the standard deviation.

In [4]:
r1 = calc_risk(port, :Trad; rm = rm)

0.3612722213729872

As a functor.

In [5]:
r1 == SD(; sigma = port.cov)(w1.weights)

true

Check that the std risk exists as an SOC constraint.

In [6]:
port.model[:sd_risk]

sd_risk

In [7]:
port.model[:constr_sd_risk_soc]

constr_sd_risk_soc : [sd_risk, 1.3339817521328776 w[1] + 0.23752111354642388 w[2] + 0.08772311968949179 w[3] + 0.1933954630317334 w[4] + 0.11104990914089145 w[5], 0.23752111354642388 w[1] + 1.1172071395407432 w[2] - 0.05472328163488465 w[3] + 0.2914549020386699 w[4] + 0.14389651700778633 w[5], 0.08772311968949179 w[1] - 0.05472328163488465 w[2] + 0.8635663605757948 w[3] - 0.00013005161391315675 w[4] - 0.4497512686528594 w[5], 0.1933954630317334 w[1] + 0.2914549020386699 w[2] - 0.00013005161391315675 w[3] + 1.4117365158022712 w[4] + 0.020326110574790338 w[5], 0.11104990914089145 w[1] + 0.14389651700778633 w[2] - 0.4497512686528594 w[3] + 0.020326110574790338 w[4] + 1.0429726582846264 w[5]] in MathOptInterface.SecondOrderCone(6)

Hierarchical risk parity optimisation, no JuMP model.

In [8]:
w2 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.199755
2,2,0.229988
3,3,0.198033
4,4,0.133306
5,5,0.238918


Compute the standard deviation.

In [9]:
r2 = calc_risk(port, :HRP; rm = rm)

0.6547632732108616

Use SD as a functor.

In [10]:
r2 == SD(; sigma = port.cov)(w2.weights)

true

## Mean Absolute Deviation, `MAD`

Vanilla mean absolute deviation.

In [11]:
rm = MAD()

MAD(RMSettings{Float64, Float64}(true, 1.0, Inf), nothing, nothing, nothing)

Optimise portfolio.

In [12]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.52867e-11
2,2,0.125902
3,3,0.558899
4,4,0.0528451
5,5,0.262353


Compute risk.

In [13]:
r1 = calc_risk(port; rm = rm)

0.2742955393862703

Check values are similar.

In [14]:
isapprox(r1, value(port.model[:mad_risk]))

true

Exponential weights.

In [15]:
ew1 = eweights(1:size(ret, 1), 0.2; scale = true);
ew2 = eweights(1:size(ret, 1), 0.3; scale = true);

Compute asset statistics, use ew1 in the `Trad` optimisation. This makes it consistent with the risk measure.

In [16]:
asset_statistics!(port; mu_type = MuSimple(; w = ew1))

Mean absolute deviation with different weights. w1 has no effect in the following optimisation in [`JuMP`](https://github.com/jump-dev/JuMP.jl)-based optimisations, so we account for it in the computation of `port.mu` above.

In [17]:
rm = MAD(; w1 = ew1, w2 = ew2)

MAD(RMSettings{Float64, Float64}(true, 1.0, Inf), [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], [0.04035360699999998, 0.05764800999999997, 0.08235429999999996, 0.11764899999999996, 0.16806999999999994, 0.24009999999999995, 0.3429999999999999, 0.48999999999999994, 0.7, 1.0], nothing)

Use the custom weights in the optimisation.

In [18]:
w2 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.0868265
2,2,1.62118e-10
3,3,0.623665
4,4,5.10551e-11
5,5,0.289508


Using `w1` and `w2` may lead to inconsistent values between the functor and value in the model because the mean absolute deviation is formulated with slack constraints.

In [19]:
r2_1 = calc_risk(port; rm = rm)

0.23722684255394683

In [20]:
r2_2 = value(port.model[:mad_risk])

0.21882857231148417

Use a custom mu (added some random noise).

In [21]:
custom_mu = port.mu + [-0.0025545471368230766, -0.0047554044723918795, 0.010574122455999866,
                       0.0021521845052968917, -0.004417767086053032]
rm = MAD(; mu = custom_mu)

MAD(RMSettings{Float64, Float64}(true, 1.0, Inf), nothing, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Optimise with the custom mu.

In [22]:
w3 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.129419
2,2,9.10067e-11
3,3,0.590633
4,4,0.13825
5,5,0.141698


Values don't match.

In [23]:
r3_1 = calc_risk(port; rm = rm)

0.359772128074709

In [24]:
r3_2 = value(port.model[:mad_risk])

0.23232767711259888

Vanilla mean absolute deviation.

In [25]:
rm = MAD()

MAD(RMSettings{Float64, Float64}(true, 1.0, Inf), nothing, nothing, nothing)

Hierarchical optimisation, no JuMP model.

In [26]:
w4 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.179264
2,2,0.222268
3,3,0.216523
4,4,0.160511
5,5,0.221434


Compute the mean absolute deviation.

In [27]:
r4 = calc_risk(port, :HRP; rm = rm)

0.46350089072293715

Use the risk measure as a functor.

In [28]:
r4 == rm(port.returns * w4.weights)

true

Custom mu has no effect in the following optimisation.

In [29]:
rm = MAD(; mu = custom_mu)

MAD(RMSettings{Float64, Float64}(true, 1.0, Inf), nothing, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Hierarchical optimisation, no JuMP model.

In [30]:
w5 = optimise!(port, HRP(; rm = rm))
w4.weights == w5.weights

true

Compute the mean absolute deviation.

In [31]:
r5 = calc_risk(port, :HRP; rm = rm)

0.46350089072293715

`w1` and `w2` both have effects.

In [32]:
rm = MAD(; w1 = ew1, w2 = ew2)

MAD(RMSettings{Float64, Float64}(true, 1.0, Inf), [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], [0.04035360699999998, 0.05764800999999997, 0.08235429999999996, 0.11764899999999996, 0.16806999999999994, 0.24009999999999995, 0.3429999999999999, 0.48999999999999994, 0.7, 1.0], nothing)

Hierarchical optimisation, no JuMP model.

In [33]:
w6 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.157301
2,2,0.179251
3,3,0.363034
4,4,0.128884
5,5,0.171529


Compute the mean absolute deviation.

In [34]:
r6 = calc_risk(port, :HRP; rm = rm)

0.3704190560125852

## Semi Standard Deviation, `SSD`

Recompute asset statistics.

In [35]:
asset_statistics!(port)

Vanilla semi standard deviation.

In [36]:
rm = SSD()

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, nothing)

Optimise portfolio.

In [37]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,3.5044e-10
2,2,2.2773e-9
3,3,0.505148
4,4,0.000939296
5,5,0.493913


Compute semi standard devation.

In [38]:
r1 = calc_risk(port; rm = rm)

0.2489122955044331

Values are consistent.

In [39]:
isapprox(r1, value(port.model[:sdev_risk]))

true

Semi standard deviation with a returns threshold equal to the maximum return, this should make it equivalent to using the standard deviation.

In [40]:
rm = SSD(; target = maximum(ret))

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 2.58911, nothing, nothing)

Optimise portfolio using the semi standard deviation with a return threshold that includes all returns.

In [41]:
w2 = optimise!(port, Trad(; obj = MinRisk(), rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,2.24485e-8
2,2,0.0183783
3,3,0.520501
4,4,0.0340455
5,5,0.427076


Optimise portfolio using the standard devation.

In [42]:
w3 = optimise!(port, Trad(; rm = SD(), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.42346e-10
2,2,0.018383
3,3,0.520498
4,4,0.0340474
5,5,0.427072


Value are approximately equal.

In [43]:
isapprox(w2.weights, w3.weights; rtol = 5e-5)

true

Exponential weights.

In [44]:
ew = eweights(1:size(ret, 1), 0.2; scale = true)

10-element Weights{Float64, Float64, Vector{Float64}}:
 0.13421772800000006
 0.1677721600000001
 0.20971520000000007
 0.2621440000000001
 0.3276800000000001
 0.4096000000000001
 0.5120000000000001
 0.6400000000000001
 0.8
 1.0

Compute asset statistics, use `ew` in the `Trad` optimisation. This makes it consistent with the risk measure.

In [45]:
asset_statistics!(port; mu_type = MuSimple(; w = ew))

Semi standard deviation with exponential weights. `w` has no effect in the following optimisation, so we account for it in the computation of `port.mu` above.

In [46]:
rm = SSD(; w = ew)

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], nothing)

Optimise using the exponential weight.

In [47]:
w4 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,9.71355e-9
2,2,2.22374e-10
3,3,0.812157
4,4,0.000746068
5,5,0.187097


Since we used the same exponential weights to compute `port.mu` and passed it on to the functor, the risk computed by `calc_risk` will be consistent with the value in the `JuMP` model.

In [48]:
r4 = calc_risk(port; rm = rm)

0.24351372267149982

Check they are approximately equal.

In [49]:
isapprox(r4, value(port.model[:sdev_risk]))

true

Custom mu (added some random noise).

In [50]:
custom_mu = port.mu + [-0.0025545471368230766, -0.0047554044723918795, 0.010574122455999866,
                       0.0021521845052968917, -0.004417767086053032]
rm = SSD(; mu = custom_mu)

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Optimise portfolio using this custom mu.

In [51]:
w5 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.69283e-8
2,2,3.85731e-10
3,3,0.805951
4,4,0.00282056
5,5,0.191229


Values don't match because the mean return is computed from the portfolio weights and returns matrix.

In [52]:
r5_1 = calc_risk(port; rm = rm)

0.34782982196502943

In [53]:
r5_2 = value(port.model[:sdev_risk])

0.24896211021424786

Vanilla semi standard deviation.

In [54]:
rm = SSD()

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, nothing)

Hierarchical optimisation, no JuMP model.

In [55]:
w6 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.184329
2,2,0.223478
3,3,0.220501
4,4,0.103132
5,5,0.268561


Compute the semi standard deviation.

In [56]:
r6 = calc_risk(port, :HRP; rm = rm)

0.5159961592965934

As a functor.

In [57]:
r6 == rm(port.returns * w6.weights)

true

Custom mu has no effect in the following optimisation.

In [58]:
rm = SSD(; mu = custom_mu)

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Hierarchical optimisation, no JuMP model.

In [59]:
w7 = optimise!(port, HRP(; rm = rm))
w6.weights == w7.weights # true

true

Compute the semi standard deviation.

In [60]:
r7 = calc_risk(port, :HRP; rm = rm)

0.5159961592965934

`w` has an effect in the following optimisation.

In [61]:
rm = SSD(; w = ew)

SSD{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], nothing)

Hierarchical optimisation, no JuMP model.

In [62]:
w8 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.152888
2,2,0.198178
3,3,0.391641
4,4,0.107353
5,5,0.149941


Compute the semi standard deviation.

In [63]:
r8 = calc_risk(port, :HRP; rm = rm)

0.46579700495675935

# First Lower Partial Moment, `FLPM`

Recompute asset statistics.

In [64]:
asset_statistics!(port)

Vanilla first lower partial moment.

In [65]:
rm = FLPM()

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, 0.0)

Optimise portfolio.

In [66]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.129097
2,2,7.13509e-10
3,3,0.586438
4,4,0.0577378
5,5,0.226727


Compute first lower partial moment.

In [67]:
r1 = calc_risk(port; rm = rm)

0.07307909070642135

Values are consistent.

In [68]:
isapprox(r1, value(port.model[:flpm_risk]))

true

First lower partial moment with a returns threshold equal to `Inf` will use `rm.mu` (which in this case is zero) in optimisations using [`JuMP`](https://github.com/jump-dev/JuMP.jl) models, and compute the mean of the returns vector when used in the functor.

In [69]:
rm = FLPM(; target = Inf)

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), Inf, nothing, 0.0)

Optimise portfolio using the first lower partial moment with a return threshold that includes all returns.

In [70]:
w2 = optimise!(port, Trad(; obj = MinRisk(), rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.129097
2,2,7.13509e-10
3,3,0.586438
4,4,0.0577378
5,5,0.226727


The risks do not match. This is because when using the functor, `mu` has no effect, and if `isinf(target)`, it will be set to the expected value of the returns vector. Whereas `PortfolioOptimiser.set_rm` took the value to be `target = range(; start = mu, stop = mu, length = N)`, where `N` is the number of assets, and `mu == 0` in this case.

In [71]:
r2_1 = calc_risk(port; rm = rm)

0.1727455718397823

In [72]:
r2_2 = value(port.model[:flpm_risk])

0.07307909079327962

If we set `rm.target = 0`, then `calc_risk` will compute the correct risk.

In [73]:
rm.target = 0
isapprox(r2_2, calc_risk(port; rm = rm))

true

First lower partial moment with a returns threshold equal to `Inf`, will use `port.mu`in optimisations using [`JuMP`](https://github.com/jump-dev/JuMP.jl) models, and compute the mean of the returns vector when used in the functor.

In [74]:
rm = FLPM(; target = Inf, mu = Inf)

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), Inf, nothing, Inf)

Value are approximately equal.

In [75]:
w3 = optimise!(port, Trad(; obj = MinRisk(), rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.8061e-11
2,2,5.26329e-10
3,3,0.591219
4,4,5.03171e-9
5,5,0.408781


Exponential weights.

In [76]:
ew = eweights(1:size(ret, 1), 0.2; scale = true)

10-element Weights{Float64, Float64, Vector{Float64}}:
 0.13421772800000006
 0.1677721600000001
 0.20971520000000007
 0.2621440000000001
 0.3276800000000001
 0.4096000000000001
 0.5120000000000001
 0.6400000000000001
 0.8
 1.0

Compute asset statistics, use `ew` in the `Trad` optimisation. This makes it consistent with the risk measure.

In [77]:
asset_statistics!(port; mu_type = MuSimple(; w = ew))

First lower partial moment with exponential weights. `w` has no effect in the following optimisation, so we account for it in the computation of `port.mu` above.

In [78]:
rm = FLPM(; w = ew)

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], 0.0)

Optimise using the exponential weight.

In [79]:
w4 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.129097
2,2,7.13509e-10
3,3,0.586438
4,4,0.0577378
5,5,0.226727


Since we used the same exponential weights to compute `port.mu` and passed it on to the functor, the risk computed by `calc_risk` will be consistent with the value in the `JuMP` model.

In [80]:
r4 = calc_risk(port; rm = rm)

0.07307909070642135

Check they are approximately equal.

In [81]:
isapprox(r4, value(port.model[:flpm_risk]))

true

Custom mu (added some random noise).

In [82]:
custom_mu = port.mu + [-0.0025545471368230766, -0.0047554044723918795, 0.010574122455999866,
                       0.0021521845052968917, -0.004417767086053032]
rm = FLPM(; mu = custom_mu)

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Optimise portfolio using this custom mu.

In [83]:
w5 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.129419
2,2,4.3467e-10
3,3,0.590633
4,4,0.13825
5,5,0.141698


Values don't match because the mean return is computed from the portfolio weights and returns matrix.

In [84]:
r5_1 = calc_risk(port; rm = rm)

0.09467150228915336

In [85]:
r5_2 = value(port.model[:flpm_risk])

0.1161638387813645

Vanilla first lower partial moment.

In [86]:
rm = FLPM()

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, 0.0)

Hierarchical optimisation, no JuMP model.

In [87]:
w6 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.222221
2,2,0.274757
3,3,0.185196
4,4,0.149013
5,5,0.168813


Compute the first lower partial moment.

In [88]:
r6 = calc_risk(port, :HRP; rm = rm)

0.17425430923284418

As a functor.

In [89]:
r6 == rm(port.returns * w6.weights)

true

Custom mu has no effect in the following optimisation.

In [90]:
rm = FLPM(; mu = custom_mu)

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Hierarchical optimisation, no JuMP model.

In [91]:
w7 = optimise!(port, HRP(; rm = rm))
w6.weights == w7.weights # true

true

If we set `target = Inf`, the target will be the return vector's expected value computed with the weights.

In [92]:
rm = FLPM(; target = Inf, w = ew)

FLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), Inf, [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], 0.0)

Hierarchical optimisation, no JuMP model.

In [93]:
w8 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.143391
2,2,0.182024
3,3,0.408805
4,4,0.163642
5,5,0.102138


Compute the first lower partial moment.

In [94]:
r8 = calc_risk(port, :HRP; rm = rm)

0.18695122450749355

# Second Lower Partial Moment, `SLPM`

Recompute asset statistics.

In [95]:
asset_statistics!(port)

Vanilla second lower partial moment.

In [96]:
rm = SLPM()

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, 0.0)

Optimise portfolio.

In [97]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.029713
2,2,0.0247062
3,3,0.513805
4,4,2.2412e-9
5,5,0.431776


Compute second lower partial moment.

In [98]:
r1 = calc_risk(port; rm = rm)

0.18612605973579038

Values are consistent.

In [99]:
isapprox(r1, value(port.model[:slpm_risk]))

true

Second lower partial moment with a returns threshold equal to `Inf` will use `rm.mu` (which in this case is zero) in optimisations using [`JuMP`](https://github.com/jump-dev/JuMP.jl) models, and compute the mean of the returns vector when used in the functor.

In [100]:
rm = SLPM(; target = Inf)

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), Inf, nothing, 0.0)

Optimise portfolio using the second lower partial moment with a return threshold that includes all returns.

In [101]:
w2 = optimise!(port, Trad(; obj = MinRisk(), rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.029713
2,2,0.0247062
3,3,0.513805
4,4,2.2412e-9
5,5,0.431776


The risks do not match. This is because when using the functor, `mu` has no effect, and if `isinf(target)`, it will be set to the expected value of the returns vector. Whereas `PortfolioOptimiser.set_rm` took the value to be `target = range(; start = mu, stop = mu, length = N)`, where `N` is the number of assets, and `mu == 0` in this case.

In [102]:
r2_1 = calc_risk(port; rm = rm)

0.26046332250133397

In [103]:
r2_2 = value(port.model[:slpm_risk])

0.18612606036834833

If we set `rm.target = 0`, then `calc_risk` will compute the correct risk.

In [104]:
rm.target = 0
isapprox(r2_2, calc_risk(port; rm = rm))

true

Second lower partial moment with a returns threshold equal to `Inf`, will use `port.mu`in optimisations using [`JuMP`](https://github.com/jump-dev/JuMP.jl) models, and compute the mean of the returns vector when used in the functor.

In [105]:
rm = SLPM(; target = Inf, mu = Inf)

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), Inf, nothing, Inf)

Value are approximately equal.

In [106]:
w3 = optimise!(port, Trad(; obj = MinRisk(), rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,3.98611e-11
2,2,9.51975e-11
3,3,0.498067
4,4,8.01089e-10
5,5,0.501933


Exponential weights.

In [107]:
ew = eweights(1:size(ret, 1), 0.2; scale = true)

10-element Weights{Float64, Float64, Vector{Float64}}:
 0.13421772800000006
 0.1677721600000001
 0.20971520000000007
 0.2621440000000001
 0.3276800000000001
 0.4096000000000001
 0.5120000000000001
 0.6400000000000001
 0.8
 1.0

Compute asset statistics, use `ew` in the `Trad` optimisation. This makes it consistent with the risk measure.

In [108]:
asset_statistics!(port; mu_type = MuSimple(; w = ew))

Second lower partial moment with exponential weights. `w` has no effect in the following optimisation, so we account for it in the computation of `port.mu` above.

In [109]:
rm = SLPM(; w = ew)

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], 0.0)

Optimise using the exponential weight.

In [110]:
w4 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.029713
2,2,0.0247062
3,3,0.513805
4,4,2.2412e-9
5,5,0.431776


Since we used the same exponential weights to compute `port.mu` and passed it on to the functor, the risk computed by `calc_risk` will be consistent with the value in the `JuMP` model.

In [111]:
r4 = calc_risk(port; rm = rm)

0.18612605973579038

Check they are approximately equal.

In [112]:
isapprox(r4, value(port.model[:slpm_risk]))

true

Custom mu (added some random noise).

In [113]:
custom_mu = port.mu + [-0.0025545471368230766, -0.0047554044723918795, 0.010574122455999866,
                       0.0021521845052968917, -0.004417767086053032]
rm = SLPM(; mu = custom_mu)

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Optimise portfolio using this custom mu.

In [114]:
w5 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.69283e-8
2,2,3.85731e-10
3,3,0.805951
4,4,0.00282056
5,5,0.191229


Values don't match because the mean return is computed from the portfolio weights and returns matrix.

In [115]:
r5_1 = calc_risk(port; rm = rm)

0.2578210994835851

In [116]:
r5_2 = value(port.model[:slpm_risk])

0.24896211021424786

Vanilla second lower partial moment.

In [117]:
rm = SLPM()

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, 0.0)

Hierarchical optimisation, no JuMP model.

In [118]:
w6 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.208317
2,2,0.266106
3,3,0.212304
4,4,0.0917449
5,5,0.221529


Compute the second lower partial moment.

In [119]:
r6 = calc_risk(port, :HRP; rm = rm)

0.39501582076391084

As a functor.

In [120]:
r6 == rm(port.returns * w6.weights)

true

Custom mu has no effect in the following optimisation.

In [121]:
rm = SLPM(; mu = custom_mu)

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.0, nothing, [0.7125701109145426, 0.5954846163775541, -0.15751039974810838, 0.21715419988378104, 0.5953722438974804])

Hierarchical optimisation, no JuMP model.

In [122]:
w7 = optimise!(port, HRP(; rm = rm))
w6.weights == w7.weights # true

true

If we set `target = Inf`, the target will be the return vector's expected value computed with the weights.

In [123]:
rm = SLPM(; target = Inf, w = ew)

SLPM{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), Inf, [0.13421772800000006, 0.1677721600000001, 0.20971520000000007, 0.2621440000000001, 0.3276800000000001, 0.4096000000000001, 0.5120000000000001, 0.6400000000000001, 0.8, 1.0], 0.0)

Hierarchical optimisation, no JuMP model.

In [124]:
w8 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.152888
2,2,0.198178
3,3,0.391641
4,4,0.107353
5,5,0.149941


Compute the second lower partial moment.

In [125]:
r8 = calc_risk(port, :HRP; rm = rm)

0.46579700495675935

# Worst Realisation, `WR`

Recompute asset statistics.

In [126]:
asset_statistics!(port)

Worst Realisation.

In [127]:
rm = WR()

WR(RMSettings{Float64, Float64}(true, 1.0, Inf))

Optimise portfolio.

In [128]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.01823e-11
2,2,5.08671e-11
3,3,0.432177
4,4,1.88254e-11
5,5,0.567823


Compute the worst realisation.

In [129]:
r1 = calc_risk(port; rm = rm)

0.3353112333705662

Values are consistent.

In [130]:
isapprox(r1, value(port.model[:wr_risk]))

true

Hierarchical optimisation, no JuMP model.

In [131]:
w2 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.179856
2,2,0.251612
3,3,0.269012
4,4,0.0620356
5,5,0.237484


Compute the worst realisation.

In [132]:
r2 = calc_risk(port, :HRP; rm = rm)

0.9545750658848196

Use it in conjunction with another, less conservative risk measure.

In [133]:
rm = [WR(; settings = RMSettings(; scale = 0.15)), Variance()]
w3 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,3.20319e-10
2,2,5.48753e-9
3,3,0.535975
4,4,6.62746e-10
5,5,0.464025


WR.

In [134]:
r3_1 = calc_risk(port; rm = WR())

0.3888545340695951

Variance.

In [135]:
r3_2 = calc_risk(port; rm = Variance())

0.1348006113596908

This portfolio is not optimal in either risk measure, but mixes their characteristics.

In [136]:
w4 = optimise!(port, Trad(; rm = Variance(), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,2.14218e-9
2,2,0.0183819
3,3,0.520499
4,4,0.0340462
5,5,0.427073


Minimum variance portfolio.

In [137]:
r4 = calc_risk(port; rm = Variance())

0.13051761827097458

WR of mixed portfolio is higher than the minimal worst realisation.

In [138]:
r3_1 > r1

true

Variance of mixed portfolio is higher than the minimal worst realisation.

In [139]:
r3_2 > r4

true

# Conditional Value at Risk, `CVaR`

Recompute asset statistics.

In [140]:
asset_statistics!(port)

CVaR with default values.

In [141]:
rm = CVaR()

CVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.05)

Optimise portfolio.

In [142]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,3.42814e-10
2,2,1.79365e-9
3,3,0.432177
4,4,7.48868e-11
5,5,0.567823


Compute CVaR for `alpha  = 0.05`.

In [143]:
r1 = calc_risk(port; rm = rm)

0.3353112360331464

Risk is consistent.

In [144]:
isapprox(r1, value(port.model[:cvar_risk]); rtol = 5e-8)

true

CVaR of the worst 50 % of cases.

In [145]:
rm = CVaR(; alpha = 0.5)

CVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.5)

Optimise portfolio.

In [146]:
w2 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.234514
2,2,0.111534
3,3,0.237635
4,4,0.278882
5,5,0.137435


Compute CVaR for `alpha  = 0.5`.

In [147]:
r2 = calc_risk(port; rm = rm)

0.03485547491800034

Values are consistent.

In [148]:
isapprox(r2, value(port.model[:cvar_risk]))

true

CVaR with default values.

In [149]:
rm = CVaR()

CVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.05)

Hierarchical optimisation, no JuMP model.

In [150]:
w3 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.179856
2,2,0.251612
3,3,0.269012
4,4,0.0620356
5,5,0.237484


Compute the CVaR.

In [151]:
r3 = calc_risk(port, :HRP; rm = rm)

0.9545750658848196

CVaR of the worst 50 % of cases.

In [152]:
rm = CVaR(; alpha = 0.5)

CVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.5)

Hierarchical optimisation, no JuMP model.

In [153]:
w4 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.187826
2,2,0.277756
3,3,0.237229
4,4,0.184567
5,5,0.112621


Compute the CVaR.

In [154]:
r4 = calc_risk(port, :HRP; rm = rm)

0.11405037467700407

# Entropic Value at Risk, `EVaR`

Recompute asset statistics.

In [155]:
asset_statistics!(port)

EVaR with default values.

In [156]:
rm = EVaR()

EVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.05, nothing)

Optimise portfolio.

In [157]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.80007e-10
2,2,2.53151e-10
3,3,0.432177
4,4,1.49677e-10
5,5,0.567823


Compute EVaR for `alpha  = 0.05`.

In [158]:
r1 = calc_risk(port; rm = rm)

0.33531123421543

As a functor, must provide the solvers.

In [159]:
rm.solvers = port.solvers
r1 == rm(port.returns * w1.weights)

true

Risk is consistent.

In [160]:
isapprox(r1, value(port.model[:evar_risk]))

true

EVaR of the worst 50 % of cases.

In [161]:
rm = EVaR(; alpha = 0.5)

EVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.5, nothing)

Optimise portfolio.

In [162]:
w2 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.0181213
2,2,0.018299
3,3,0.507395
4,4,1.79592e-9
5,5,0.456185


Compute EVaR for `alpha  = 0.5`.

In [163]:
r2 = calc_risk(port; rm = rm)

0.26782403994246134

Values are consistent.

In [164]:
isapprox(r2, value(port.model[:evar_risk]))

true

EVaR with default values.

In [165]:
rm = EVaR()

EVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.05, nothing)

Hierarchical optimisation, no JuMP model but needs solvers.

In [166]:
w3 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.179856
2,2,0.251612
3,3,0.269012
4,4,0.0620356
5,5,0.237484


Compute the EVaR.

In [167]:
r3 = calc_risk(port, :HRP; rm = rm)

0.954575073881352

EVaR of the worst 50 % of cases.

In [168]:
rm = EVaR(; alpha = 0.5)

EVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.5, nothing)

Hierarchical optimisation, no JuMP model.

In [169]:
w4 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.210601
2,2,0.281815
3,3,0.21888
4,4,0.0884636
5,5,0.20024


Compute the EVaR.

In [170]:
r4 = calc_risk(port, :HRP; rm = rm)

0.4995910664303083

# Relativistic Value at Risk, `RLVaR`

Recompute asset statistics.

In [171]:
asset_statistics!(port)

RLVaR with default values.

In [172]:
rm = RLVaR()

RLVaR{Float64, Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.05, 0.3, nothing)

Optimise portfolio.

In [173]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.7216e-10
2,2,3.57343e-11
3,3,0.432177
4,4,1.98357e-10
5,5,0.567823


Compute RLVaR for `alpha  = 0.05`.

In [174]:
r1 = calc_risk(port; rm = rm)

0.33531123214615577

As a functor, must provide the solvers.

In [175]:
rm.solvers = port.solvers
r1 == rm(port.returns * w1.weights)

true

Risk is consistent.

In [176]:
isapprox(r1, value(port.model[:rlvar_risk]))

true

RLVaR of the worst 50 % of cases.

In [177]:
rm = RLVaR(; alpha = 0.5)

RLVaR{Float64, Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.5, 0.3, nothing)

Optimise portfolio.

In [178]:
w2 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.000193217
2,2,7.46254e-9
3,3,0.487596
4,4,5.06905e-10
5,5,0.512211


Compute RLVaR for `alpha  = 0.5`.

In [179]:
r2 = calc_risk(port; rm = rm)

0.278580947684099

Values are consistent.

In [180]:
isapprox(r2, value(port.model[:rlvar_risk]))

true

Check the limits as `kappa → 0`, and `kappa → Inf`. We use a large value of alpha because there are very few observations, so we need it to differentiate the results of the optimisations.

In [181]:
w3_1 = optimise!(port, Trad(; rm = RLVaR(; alpha = 0.5, kappa = 5e-5), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.0181197
2,2,0.0182987
3,3,0.507394
4,4,2.95838e-8
5,5,0.456187


In [182]:
w3_2 = optimise!(port,
                 Trad(; rm = RLVaR(; alpha = 0.5, kappa = 1 - 5e-5), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,4.13117e-14
2,2,3.08176e-14
3,3,0.432177
4,4,5.61983e-14
5,5,0.567823


In [183]:
w3_3 = optimise!(port, Trad(; rm = EVaR(; alpha = 0.5), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.0181213
2,2,0.018299
3,3,0.507395
4,4,1.79592e-9
5,5,0.456185


In [184]:
w3_4 = optimise!(port, Trad(; rm = WR(), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,1.01823e-11
2,2,5.08671e-11
3,3,0.432177
4,4,1.88254e-11
5,5,0.567823


$\\lim\\limits_{\\kappa \\to 0} \\mathrm{RLVaR}(\\bm{X},\\, \\alpha,\\, \\kappa) \\approx \\mathrm{EVaR}(\\bm{X},\\, \\alpha)$

In [185]:
rmsd(w3_1.weights, w3_3.weights)

1.2142341888644785e-6

$\\lim\\limits_{\\kappa \\to 1} \\mathrm{RLVaR}(\\bm{X},\\, \\alpha,\\, \\kappa) \\approx \\mathrm{WR}(\\bm{X})$

In [186]:
rmsd(w3_2.weights, w3_4.weights)

4.790216951410363e-11

RLVaR with default values.

In [187]:
rm = RLVaR()

RLVaR{Float64, Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.05, 0.3, nothing)

Hierarchical optimisation, no JuMP model but needs solvers.

In [188]:
w4 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.179856
2,2,0.251612
3,3,0.269012
4,4,0.0620356
5,5,0.237484


Compute the RLVaR.

In [189]:
r4 = calc_risk(port, :HRP; rm = rm)

0.9545750670771044

RLVaR of the worst 50 % of cases.

In [190]:
rm = EVaR(; alpha = 0.5)

EVaR{Float64}(RMSettings{Float64, Float64}(true, 1.0, Inf), 0.5, nothing)

Hierarchical optimisation, no JuMP model.

In [191]:
w5 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.210601
2,2,0.281815
3,3,0.21888
4,4,0.0884636
5,5,0.20024


Compute the RLVaR.

In [192]:
r5 = calc_risk(port, :HRP; rm = rm)

0.4995910664303083

# Maximum Drawdown of Uncompounded Cumulative Returns, `MDD`

Recompute asset statistics.

In [193]:
asset_statistics!(port)

Maximum drawdown of uncompounded returns.

In [194]:
rm = MDD()

MDD(RMSettings{Float64, Float64}(true, 1.0, Inf))

Optimise portfolio.

In [195]:
w1 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.13391
2,2,1.17101e-11
3,3,0.496406
4,4,8.35795e-12
5,5,0.369684


Compute MDD.

In [196]:
r1 = calc_risk(port; rm = rm)

0.5374188549183612

Values are consistent.

In [197]:
isapprox(r1, value(port.model[:mdd_risk]))

true

Hierarchical optimisation, no JuMP model.

In [198]:
w2 = optimise!(port, HRP(; rm = rm))

Row,tickers,weights
,Int64,Float64
1,1,0.340287
2,2,0.254799
3,3,0.132603
4,4,0.124343
5,5,0.147968


Compute the MDD.

In [199]:
r2 = calc_risk(port, :HRP; rm = rm)

1.2557175411222494

Use it in conjunction with another, less conservative risk measure.

In [200]:
rm = [MDD(; settings = RMSettings(; scale = 0.15)), Variance()]
w3 = optimise!(port, Trad(; rm = rm, str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,0.024221
2,2,4.10595e-9
3,3,0.520211
4,4,0.0302782
5,5,0.42529


MDD.

In [201]:
r3_1 = calc_risk(port; rm = MDD())

0.7055289743577166

Variance.

In [202]:
r3_2 = calc_risk(port; rm = Variance())

0.13551503837607193

This portfolio is not optimal in either risk measure, but mixes their characteristics.

In [203]:
w4 = optimise!(port, Trad(; rm = Variance(), str_names = true))

Row,tickers,weights
,Int64,Float64
1,1,2.14218e-9
2,2,0.0183819
3,3,0.520499
4,4,0.0340462
5,5,0.427073


Minimum variance portfolio.

In [204]:
r4 = calc_risk(port; rm = Variance())

0.13051761827097458

MDD of mixed portfolio is higher than the minimal MDD.

In [205]:
r3_1 > r1

true

Variance of mixed portfolio is higher than the minimal MDD.

In [206]:
r3_2 > r4

true

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*